In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1234) 

In [2]:
index_char = ['h','i','e','l','o']
x_data = [[0,1,0,2,3,3]] #IHELLO
x_one_hot = [[[1,0,0,0,0], [0,1,0,0,0], [1,0,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,1,0]]]
print(x_data)
print(x_one_hot)

[[0, 1, 0, 2, 3, 3]]
[[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0]]]


In [3]:
y_data = [[1,0,2,3,3,4]] # i hello

In [4]:
number_classes = 5
input_dimen = 5 # one_hot size
hidden_size = 5
batch_size = 1
sequence_length = 6

In [5]:
x = tf.placeholder(tf.float32, [None,sequence_length,input_dimen]) # [None, 6, 5]
y = tf.placeholder(tf.int32, [None, sequence_length]) # [None, 6]

In [6]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
init_state = cell.zero_state(batch_size, tf.float32)

In [7]:
output, _state = tf.nn.dynamic_rnn(cell, x, initial_state=init_state, dtype=tf.float32)

In [8]:
x_fc = tf.reshape(output, [-1, hidden_size])
output = tf.contrib.layers.fully_connected(inputs=x_fc, num_outputs=number_classes, activation_fn=None)
output = tf.reshape(output, [batch_size, sequence_length, number_classes])

In [9]:
weight = tf.ones([batch_size,sequence_length])
print(weight)

Tensor("ones:0", shape=(1, 6), dtype=float32)


In [10]:
seq_loss = tf.contrib.seq2seq.sequence_loss(logits=output, targets=y, weights=weight)
loss = tf.reduce_mean(seq_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(output, axis = 2)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l,_ = sess.run([loss,train], feed_dict={x:x_one_hot, y:y_data})
        result = sess.run(prediction, feed_dict={x:x_one_hot})
        
        if i % 10 == 0:
            print(i,'loss:',l,"prediction:", result, 'T_y :', y_data)
            resurt_string = [index_char[c] for c in np.squeeze(result)]
            print(''.join(resurt_string))


0 loss: 1.63581 prediction: [[3 3 3 3 3 3]] T_y : [[1, 0, 2, 3, 3, 4]]
llllll
10 loss: 0.689607 prediction: [[2 0 2 3 3 4]] T_y : [[1, 0, 2, 3, 3, 4]]
ehello
20 loss: 0.230575 prediction: [[1 0 2 3 3 4]] T_y : [[1, 0, 2, 3, 3, 4]]
ihello
30 loss: 0.0259902 prediction: [[1 0 2 3 3 4]] T_y : [[1, 0, 2, 3, 3, 4]]
ihello
40 loss: 0.00493595 prediction: [[1 0 2 3 3 4]] T_y : [[1, 0, 2, 3, 3, 4]]
ihello
